# Matrixsteifigkeitsmethode: 2D-Fachwerk

In diesem Notebook baust du dir einen kleinen Fachwerk-Löser. Nicht weil du später alles von Hand programmieren musst, sondern weil man dabei sehr gut sieht, **was die Methode wirklich macht**.

Heute schauen wir nur auf:
- **Verschiebungen** $U$
- **Kräfte** $F$ (inkl. Reaktionen an den gelagerten Freiheitsgraden)

Dehnungen und Spannungen lassen wir bewusst weg. Die kommen erst, wenn die Grundmaschine sauber läuft.


## Wie du hier vorgehst


In den Zellen findest du Platzhalter wie:

- `# TODO: ...`
- `...` (drei Punkte)

Du sollst jeweils **nur den Ausdruck rechts** einsetzen, z.B.:
```python
k_local = "hier entsprechende Formel einsetzen"
```

Wir gehen von klein nach gross:
1. Elementebene: 
    - $k_{local}$: lokale Elementsteifigkeitsmatrix, 
    - $T$: Transformationsmatrix 
    - $K_e$: globale Elementsteifigketismatrix
2. Strukturebene: 
    - Inzidenztabelle 
    - $K$: Aus Elementsteifigkeitsmatrizen assemblierte globale Struktursteifigkeitsmatrix
3. Randbedingungen: 
    - eine Zeile für $U_F$


## Imports

Wir brauchen nur NumPy.


In [ ]:
import numpy as np


## Modell

Das ist unser Beispiel-Fachwerk. Die Daten sind fertig, damit du dich auf die Methode konzentrieren kannst.


In [ ]:
# Knotenkoordinaten [X, Y] in m
nodal_coordinates = np.array([
    [0.0, 0.0],
    [1.0, 0.0],
    [1.0, 1.0],
    [2.0, 1.0],
])

# Elemente: [Knoten_i, Knoten_j, section_key]
elements = [
    [0, 1, "section 1"],
    [0, 2, "section 2"],
    [1, 2, "section 3"],
    [1, 3, "section 4"],
    [2, 3, "section 3"],
]

# Materialien: Youngscher Modul [Pa]
materials = {"steel": [210e9]}

# Querschnitte: [A, material_key]
sections = {
    "section 1": [15e-6,    "steel"],
    "section 2": [28.28e-6, "steel"],
    "section 3": [10e-6,    "steel"],
    "section 4": [56.58e-6, "steel"],
}

# Randbedingungen: [Knoten, Achse (0=x,1=y), vorgegebene Verschiebung]
constraints = [
    [0, 0, 0.0],
    [0, 1, 0.0],
    [1, 1, 0.0],
]

# Lasten: [Knoten, Achse (0=x,1=y), Kraft]
loads = [
    [3, 1, -1e3],
]


## 1) Elementebene: $k_{local}$, $T$, $K_e$

Hier ist alles schon als Code-Gerüst vorhanden. Du füllst nur die Ausdrücke bei den `...`.

Formeln aus der Vorlesung:

- $k_{local} = \frac{EA}{L}\begin{bmatrix}1&-1\\-1&1\end{bmatrix}$
- $u_{\parallel} = c u_x + s u_y$
- $K_e = T^T k_{local} T$

Hinweis: `T` ist (2x4). Erste Zeile nimmt die Parallelkomponente am Knoten i, zweite Zeile am Knoten j.


In [ ]:
def element_stiffness_matrix(EA, xy_e):
    """Globale Elementsteifigkeitsmatrix Ke für ein 2D-Stab-Element."""
    dx = xy_e[1,0] - xy_e[0,0]
    dy = xy_e[1,1] - xy_e[0,1]
    L  = np.sqrt(dx**2 + dy**2)

    c = dx / L
    s = dy / L

    # TODO: lokale Elementsteifigkeit (2x2)
    k_local = ...

    # TODO: Transformationsmatrix (2x4)
    T = ...

    # TODO: globale Elementmatrix (4x4)
    Ke = ...

    return Ke


## 2) Inzidenztabelle (DOF-Mapping)

Der folgende Code erzeugt die **Inzidenztabelle**. Sie beschreibt, **an welchen Positionen in der globalen Struktursteifigkeitsmatrix** die Einträge der jeweiligen Elementsteifigkeitsmatrix abgelegt werden.
Das funktioniert so:
- Aus den Elementen werden die Knotenpaare `(i, j)` extrahiert.
- Jeder Knoten hat zwei Freiheitsgrade: `ux = 2*i`, `uy = 2*i + 1`.
- Für jedes Element entsteht ein Vektor `[2*i, 2*i+1, 2*j, 2*j+1]`.


In [ ]:
def incidence_table(elements):
    # Nur Knotenpaare extrahieren (i, j)
    conn = np.array([[e[0], e[1]] for e in elements], dtype=int)
    dofs = np.vstack((
        2*conn[:,0],
        2*conn[:,0] + 1,
        2*conn[:,1],
        2*conn[:,1] + 1,
    )).T
    return dofs


## 3) Assemblierung: globale Struktursteifigkeit $K$

Diese Funktion ist fertig. Sie nimmt deine `incidence_table` und `element_stiffness_matrix` und baut daraus $K$.


In [ ]:
def assemble_K(nodal_coordinates, elements, sections, materials):
    dofs = incidence_table(elements)
    ndof = int(np.max(dofs) + 1)
    K = np.zeros((ndof, ndof))

    for e, (i, j, sec_key) in enumerate(elements):
        A, mat_key = sections[sec_key]
        E = materials[mat_key][0]
        EA = E * A

        xy_e = nodal_coordinates[[i, j], :]
        Ke = element_stiffness_matrix(EA, xy_e)

        edofs = dofs[e]
        K[np.ix_(edofs, edofs)] += Ke

    return K


## 4) Randbedingungen und Lösen

Auch hier steht fast alles. Du füllst nur **eine** Zeile:

$$
U_F = K_{FF}^{-1}(F_F - K_{FU}U_U)
$$

In NumPy:
`np.linalg.solve(K_FF, rhs)` löst $K_{FF} x = rhs$.


In [ ]:
def solve_system(K, constraints, loads):
    ndof = K.shape[0]

    _U = np.zeros(ndof, dtype=bool)
    U_U = []

    for node, axis, val in constraints:
        dof = 2*int(node) + int(axis)
        _U[dof] = True
        U_U.append(val)

    U_U = np.array(U_U, dtype=float)
    _F = ~_U

    F = np.zeros(ndof)
    for node, axis, val in loads:
        dof = 2*int(node) + int(axis)
        F[dof] = val

    K_FF = K[np.ix_(_F, _F)]
    K_FU = K[np.ix_(_F, _U)]
    F_F  = F[_F]

    # TODO: freie Verschiebungen
    U_F = ...

    U = np.zeros(ndof)
    U[_U] = U_U
    U[_F] = U_F

    # Reaktionen
    K_UF = K[np.ix_(_U, _F)]
    K_UU = K[np.ix_(_U, _U)]
    F_U = K_UF @ U_F + K_UU @ U_U
    F[_U] = F_U

    return U, F, _U


## Ausführen

Wenn du die `...` ersetzt hast, kannst du rechnen.

Mini-Check für dich:
- `K` muss symmetrisch sein.
- an festen DOFs sind Verschiebungen 0.


In [ ]:
K = assemble_K(nodal_coordinates, elements, sections, materials)
U, F, fixed_mask = solve_system(K, constraints, loads)

np.set_printoptions(precision=6, suppress=True)
print("Verschiebungen U [ux0, uy0, ux1, uy1, ...]:")
print(U)

print("\nKraefte F (inkl. Reaktionen an festen DOFs):")
print(F)

print("\nSymmetrie-Check K (sollte nahe 0 sein):", np.max(np.abs(K - K.T)))
